Read the first 2 pages only: https://www.cmi.ac.in/~madhavan/courses/datamining12/reading/em-tutorial.pdf

Why/Use: A short friendly introduction to EM. You should understand the coin-flipping example, identify the data, parameters, and latents, and the E and M steps. At this point, don't worry if you don't fully understand why the algorithm works.

Read the whole section called “Further Discussion”:
https://en.wikipedia.org/wiki/Variational_Bayesian_methods#Further_discussion

Why/Use: Make sure you understand all the 9 steps in the first subsection, relating each of them to the “Mean-Field Approximation for the Univariate Gaussian” example from last week’s reading (http://bjlkeng.github.io/posts/variational-bayes-and-the-mean-field-approximation/). Then focus on the similarities and differences between Variational Bayes and EM, presented in the last subsection “Compared with expectation maximization (EM)”.

The following is partly adapted from Wikipedia: https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm. You may want to refer to the article while reading the derivations below:

#### Expectation-Maximization basics:

We want to find a set of parameters that maximize the likelihood of the data: $\underset{\theta}{\mathrm{argmax}}p(X|\theta)$. However, $p(X|\theta)=\int p(X,Z|\theta)dZ$ for some latent variables $Z$ (if $Z$ is discrete, it is a sum). Calculating this marginal is often intractable.

#### EM as coordinate ascent variational inference (for discrete $Z$):

From VI, we know:
$$\log{p(x)}\geq \log{p(x)}-D_{KL}(q(z)||p(z|x))=E_{q(z)}[\log{p(x)}]-E_{q(z)}[\log{q(z)}]+E_{q(z)}[\log{p(z|x)}]=E_{q(z)}\log{p(z,x)}-E_{q(z)}[\log{q(z)}]=-E_{q(z)}[\log{\dfrac{q(z)}{p(z,x)}}]$$

In other words, if we need a lower bound for $\log{p(x)}$, it is natural to write $-E_{q(z)}[\log{\dfrac{q(z)}{p(z,x)}}]$.

In EM, we need to find a function of $\theta$ that is a lower bound to the likelihood $p(X|\theta)$. Then we can find the value of the parameters for which this lower bound is maximized, instead of finding the parameters that maximize $p(X|\theta)$. This new value of the parameters may not maximize $p(X|\theta)$, but it at least increases it. So repeating the process of finding new lower bounds, and then maximizing them brings incremental increases to $p(X|\theta)$, until it reaches a local maximum.

<img src="EM_1.png" width="300"/>

As stated above, it is natural to write $-E_{q(Z)}[\log{\dfrac{q(Z)}{p(Z,X|\theta)}}]$ if we want a lower bound for $\log{p(X|\theta)}$.

Let $q(Z)$ be an arbitrary probability distribution of $Z$. Let $\theta^{(t)}$ be a fixed value for the of parameters. Consider the following functional of $q(Z)$:
$$F(q(Z),\theta^{(t)})=-E_{q(Z)}[\log{\dfrac{q(Z)}{p(Z,X|\theta^{(t)})}}]$$

##### E-step:

In the E-step, we're going to find the distribution $q^{(t)}(Z)$ that maximizes this functional:
$$q^{(t)}(Z)=\underset{q(Z)}{\mathrm{argmax}}F(q(Z),\theta^{(t)}).$$

We have:
$$F(q(Z),\theta^{(t)})=-E_{q(Z)}[\log{\dfrac{q(Z)}{p(Z,X|\theta^{(t)})}}]=E_{q(Z)}[\log{p(X,Z|\theta^{(t)})}]-E_{q(Z)}[\log{q(Z)}]=\sum_Zq(Z)\log{p(X,Z|\theta^{(t)})}-\sum_Zq(Z)\log{q(Z)}$$
$$=\sum_Zq(Z)\log{p(Z|X,\theta^{(t)})}+\sum_Zq(Z)\log{p(X|\theta^{(t)})}-\sum_Zq(Z)\log{q(Z)}=\sum_Zq(Z)\log{\dfrac{p(Z|X,\theta^{(t)})}{q(Z)}}+\log{p(X|\theta^{(t)})}\sum_Zq(Z)$$
$$=-D_{KL}(q(Z)||p(Z|X,\theta^{(t)}))+\log{p(X|\theta^{(t)})}=-D_{KL}(q(Z)||p(Z|X,\theta^{(t)}))+const$$

Hence chosing $q(Z)$ that maximizes $F(q(Z),\theta^{(t)})$ is equivalent to choosing $q(Z)$ that minimizes $D_{KL}(q(Z)||p(Z|X,\theta^{(t)}))$, which is, of course, $q^{(t)}(Z)=p(Z|X,\theta^{(t)})$. For $q(Z)=q^{(t)}(Z)$, the functional takes the value $F(p(Z|X,\theta^{(t)}),\theta^{(t)})$. Therefore, we would expect that $F(p(Z|X,\theta^{(t)}),\theta)$, as a function of $\theta$, will be a lower bound for $\log{p(X|\theta)}$. (we will prove this later)

##### M-step:

In the M-step, we will find the optimal value for the parameters $\theta$ that maximizes this function of $\theta$: $F(p(Z|X,\theta^{(t)}),\theta)$. So let
$$\theta^{(t+1)}=\underset{\theta}{\mathrm{argmax}}F(p(Z|X,\theta^{(t)}),\theta)=\underset{\theta}{\mathrm{argmax}}-E_{p(Z|X,\theta^{(t)})}[\log{\dfrac{p(Z|X,\theta^{(t)})}{p(Z,X|\theta)}}]=\underset{\theta}{\mathrm{argmax}}E_{p(Z|X,\theta^{(t)})}[\log{p(Z,X|\theta)}]$$

The last equality is because $E_{p(Z|X,\theta^{(t)})}[\log{p(Z|X,\theta^{(t)})}]$ does not depend on $\theta$.

Setting $Q(\theta|\theta^{(t)})=E_{p(Z|X,\theta^{(t)})}[\log{p(Z,X|\theta)}]$, we get $\theta^{(t+1)}=\underset{\theta}{\mathrm{argmax}}Q(\theta|\theta^{(t)})$.

##### Proof of correctness:

Now, we need to prove two facts:

Fact 1: $\log{p(X|\theta^{(t+1)})}\geq \log{p(X|\theta^{(t)})}$, i.e. the M-step never decreases the log likelihood.

Fact 2: $F(p(Z|X,\theta^{(t)}),\theta)$ is a lower bound of the log likelihood $\log{p(X|\theta)}$, as functions of $\theta$.

We have:

$$Q(\theta|\theta^{(t)})=E_{p(Z|X,\theta^{(t)})}[\log{p(Z,X|\theta)}]=\sum_{Z}p(Z|X,\theta^{(t)})\log{p(X,Z|\theta)}=\sum_{Z}p(Z|X,\theta^{(t)})\log{p(Z|X,\theta)}+\sum_{Z}p(Z|X,\theta^{(t)})\log{p(X|\theta)}$$
$$=\sum_{Z}\Big(p(Z|X,\theta^{(t)})\log{p(Z|X,\theta)}+p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}-p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}\Big)+\log{p(X|\theta)}\sum_{Z}p(Z|X,\theta^{(t)})$$
$$=\sum_{Z}\Big(-p(Z|X,\theta^{(t)})\log{\dfrac{p(Z|X,\theta^{(t)})}{p(Z|X,\theta)}}+p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}\Big)+\log{p(X|\theta)}$$
$$=-\sum_{Z}p(Z|X,\theta^{(t)})\log{\dfrac{p(Z|X,\theta^{(t)})}{p(Z|X,\theta)}}+\sum_{Z}p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}+\log{p(X|\theta)}$$
$$=-D_{KL}(p(Z|X,\theta^{(t)})||p(Z|X,\theta))+\log{p(X|\theta)}+C\leq \log{p(X|\theta)}+C$$

where equality is obtained when the two distributions in the KL divergence are equal, i.e. when $p(Z|X,\theta)=p(Z|X,\theta^{(t)})$, or when $\theta=\theta^{(t)}$. Here $C=\sum_{Z}p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}$ does not depend on $\theta$, so it is a constant. Hence we got the following:

$$Q(\theta|\theta^{(t)})\leq \log{p(X|\theta)}+C$$
$$Q(\theta^{(t)}|\theta^{(t)}) = \log{p(X|\theta^{(t)})}+C$$

Subtracting the equality from the inequality, we obtain:
$$Q(\theta|\theta^{(t)})-Q(\theta^{(t)}|\theta^{(t)})\leq \log{p(X|\theta)}-\log{p(X|\theta^{(t)})}$$

By the definition of $\theta^{(t+1)}$, we have $Q(\theta^{(t+1)}|\theta^{(t)})\geq Q(\theta^{(t)}|\theta^{(t)})$. So from this and the result above, we get:
$$0\leq Q(\theta^{(t+1)}|\theta^{(t)})-Q(\theta^{(t)}|\theta^{(t)})\leq \log{p(X|\theta^{(t+1)})}-\log{p(X|\theta^{(t)})}$$
$$\log{p(X|\theta^{(t+1)})}\geq \log{p(X|\theta^{(t)})}$$
which proves Fact 1.

To prove Fact 2, we use that
$$Q(\theta|\theta^{(t)})\leq \log{p(X|\theta)}+C$$
$$Q(\theta|\theta^{(t)})-C\leq \log{p(X|\theta)}$$
$$E_{p(Z|X,\theta^{(t)})}[\log{p(Z,X|\theta)}]-\sum_{Z}p(Z|X,\theta^{(t)})\log{p(Z|X,\theta^{(t)})}\leq \log{p(X|\theta)}$$
$$E_{p(Z|X,\theta^{(t)})}[\log{p(Z,X|\theta)}]-E_{p(Z|X,\theta^{(t)})}[\log{p(Z|X,\theta^{(t)})}]\leq \log{p(X|\theta)}$$
$$-E_{p(Z|X,\theta^{(t)})}[\log{\dfrac{p(Z|X,\theta^{(t)})}{p(Z,X|\theta)}}]\leq \log{p(X|\theta)}$$
$$F(p(Z|X,\theta^{(t)}),\theta)\leq \log{p(X|\theta)}$$

Which proves Fact 2.

Now read all sections from https://en.wikipedia.org/wiki/Expectation–maximization_algorithm except "Filtering and smoothing EM algorithms". In particular, focus on the "Gaussian mixture" example. Try to understand the example as fully as possible.

(Optional) Read the discussion about the correspondence to EM in the final paragraph of the “A more complex example” section in https://en.wikipedia.org/wiki/Variational_Bayesian_methods#A_more_complex_example. While you read, refer to the EM solution to the Mixture of Gaussians problem: https://en.wikipedia.org/wiki/Mixture_model#Expectation_maximization_(EM).

(Optional) Pre-class work: Implement the "Gaussian mixture" example from https://en.wikipedia.org/wiki/Expectation–maximization_algorithm in python.